# Interactive Tile

> TODO: describe the tile

In [1]:
%matplotlib tk

from sorbetto.core.entity import Entity
from sorbetto.performance.two_class_classification_performance import TwoClassClassificationPerformance
from sorbetto.ranking.ranking_score import RankingScore
from sorbetto.parameterization import ParameterizationDefault
from sorbetto.core.importance import Importance
from sorbetto.flavor.value_flavor import ValueFlavor
from sorbetto.tile.value_tile import ValueTile
from sorbetto.tile.tile import Tile

import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
from matplotlib.widgets import Slider

import numpy as np

import os
import json


def get_colors ( num_colors ) :
    x = np.linspace ( 0.0 , 1.0 , num_colors )
    a = np.floor ( x * x * np.sqrt ( num_colors ) )
    v = 1.0 - a / ( np.max ( a ) + 1 )
    h = x * x * ( np.max ( a ) + 1 )
    h = h - np.floor ( h )
    hsv = np.ones ( [num_colors,3] )
    hsv [:,0] = h
    hsv [:,1] = v
    hsv [:,2] = 1.0
    rgb = hsv_to_rgb( hsv )
    rgba = np.ones ( [num_colors,4] )
    rgba [:,0:3] = rgb
    return rgba


In [2]:
# loading data

PATH_DATA = os.path.join(os.getcwd(), 'data', 'semantic_segmentation_methods.json')

MAP_DATASETS = {
    'Cityscapes': '♠',
    'ADE20K': '♥',
    'Pascal VOC 2012': '♦',
    'COCO-Stuff 164k': '♣'
}

def read_json(filePath:str):
    """
    Function to read json 
    """
    
    with open(filePath, 'r') as f:
        data = json.load(f)
    return data


def read_data(filePath:str = PATH_DATA):
    data = read_json(filePath)

    processed_data = []

    colors = get_colors(len(data))

    # convert each instance into an Entity
    for i, elem in enumerate(data):
        performance = TwoClassClassificationPerformance(
            ptp=elem['proba_tp'],
            ptn=elem['proba_tn'],
            pfp=elem['proba_fp'],
            pfn=elem['proba_fn']
        )

        entity = Entity(
            performance= performance,
            name = f"{elem['model']} {MAP_DATASETS[elem['learning_set']]}",
            color= colors[i]
        )

        processed_data.append(entity)

    return processed_data



data = read_data()

In [3]:
id_entity = 0

perf = data[id_entity].performance

In [4]:
parameterization = ParameterizationDefault ()
min_x, max_x = parameterization.getBoundsParameter1 ()
min_y, max_y = parameterization.getBoundsParameter2 ()
extent = [ min_x, max_x, min_y, max_y ]

In [5]:
click_marker_left = [None]
click_marker_right = [None]

In [6]:
fig, ax = plt.subplots(1, 2, layout="constrained")
tile = Tile(name="Empty tile", parameterization = parameterization)
tile.draw(fig, ax[0])
right_spec = ax[1].get_subplotspec()
left_spec = ax[0].get_subplotspec()

click_marker_left = [None]
click_marker_right = [None]

def reset_marker(marker):
    if marker[0] is not None:
        marker[0].remove()
        marker[0] = None


def showRankingScore(score, fig, ax, priorPos = [.5]):
    for priorPos in priorPos:
        score.drawInROC(fig, ax, priorPos=priorPos)

def get_ranking_score(a, b):
    itp = a
    itn = 1-a
    ifp = 1-b
    ifn = b
    importance = Importance(itn=itn, ifp=ifp, ifn=ifn, itp=itp)
    return RankingScore(importance=importance)

def plot_roc(fig, ax, coords):
    importance = parameterization.getCanonicalImportance(*coords)
    rankScore = RankingScore(importance=importance)
    showRankingScore(rankScore, fig, ax)
    ax.set_box_aspect(1)
    ax.set_anchor('C')
    ax.axis('off')
    ax.set_title('ROC', fontsize=12, pad=20)


def plot_tile(fig, ax, tpr, fpr, priorPos = 0.5):
    
    fnr = 1-tpr
    tnr = 1-fpr

    priorNeg = 1-priorPos

    ptn = tnr * priorNeg
    pfp = fpr * priorNeg
    pfn = fnr * priorPos
    ptp = tpr * priorPos

    perf = TwoClassClassificationPerformance(ptn = ptn,
                                            pfp=pfp,
                                            pfn = pfn,
                                            ptp = ptp)
    v_flavor = ValueFlavor("Value flavor")

    v_tile = ValueTile(name = "Value Tile", 
                       parameterization = parameterization, 
                       flavor = v_flavor,
                       performance=perf
                       )
    
    a = np.linspace(0, 1, 100)
    b = np.linspace(0, 1, 100)

    a2, b2 = np.meshgrid(a, b, indexing='xy')
    

    res = v_tile(a2, b2)

    ax.imshow(res, extent=extent, origin='lower')
    ax.set_title('Value Tile', fontsize=12, pad=20)

def react_left(fig, ax, event, click_marker_left = click_marker_left, click_marker_right = click_marker_right):

    fig.delaxes(ax[1])
    ax[1] = fig.add_subplot(right_spec)

    plot_roc(fig, ax[1], (event.xdata, event.ydata))

    print("-"*10)
    print(click_marker_left)
    reset_marker(click_marker_left)
    print(click_marker_left)
    print("-"*10)

    click_marker_left[0] = ax[0].plot(event.xdata, event.ydata, 'ro', markersize=10)[0]
    if click_marker_right[0] is not None:
        click_marker_right[0] = ax[1].plot(click_marker_right[0].get_xdata()[0], click_marker_right[0].get_ydata()[0], 'ro', markersize=10)[0]

    fig.canvas.draw_idle()



def react_right(fig, ax, event, click_marker_left = click_marker_left, click_marker_right = click_marker_right):

    fig.delaxes(ax[0])
    ax[0] = fig.add_subplot(left_spec)

    plot_tile(fig, ax[0], event.ydata, event.xdata, priorPos = 0.5)

    print("*"*10)
    print(click_marker_right)
    reset_marker(click_marker_right)
    print(click_marker_right)
    print("*"*10)

    click_marker_right[0] = ax[1].plot(event.xdata, event.ydata, 'ro', markersize=10)[0]
    if click_marker_left[0] is not None:
        click_marker_left[0] = ax[0].plot(click_marker_left[0].get_xdata()[0], click_marker_left[0].get_ydata()[0], 'ro', markersize=10)[0]

    fig.canvas.draw_idle()

    print(click_marker_right)
    print(click_marker_left)
    
    

def del_colorbars(fig):
    for a in list(fig.axes):
        if a.get_label() == '<colorbar>':
            fig.delaxes(a)

def on_move(event):
    # Only react when the mouse is inside the left axes and has valid data coords
    if event.xdata is None or event.ydata is None:
        return
    
    elif event.inaxes is not ax[0]:
        
        react_right(fig, ax, event)

    else:
        react_left(fig, ax, event)



# Optional: disable updates during pan/zoom
def toolbar_busy():
    tb = getattr(fig.canvas, "toolbar", None)
    return bool(tb and tb.mode)  # don't update while zoom/pan active

def on_move_guarded(event):
    if not toolbar_busy():
        on_move(event)

#cid_move = fig.canvas.mpl_connect('motion_notify_event', on_move_guarded)

plt.connect('button_press_event', on_move_guarded)


plt.show()


In [7]:

fig, ax = plt.subplots(1, 2, layout="constrained")
tile = Tile(name="Empty tile", parameterization = parameterization)
tile.draw(fig, ax[0])
right_spec = ax[1].get_subplotspec()
left_spec = ax[0].get_subplotspec()

click_marker_left = [None]
click_marker_right = [None]
priorPos_slider_value = [0.1]

def reset_marker(marker):
    if marker[0] is not None:
        marker[0].remove()
        marker[0] = None


def showRankingScore(score, fig, ax):
    priorPos = priorPos_slider_value[0]
    score.drawInROC(fig, ax, priorPos=priorPos)

def get_ranking_score(a, b):
    itp = a
    itn = 1-a
    ifp = 1-b
    ifn = b
    importance = Importance(itn=itn, ifp=ifp, ifn=ifn, itp=itp)
    return RankingScore(importance=importance)

def plot_roc(fig, ax, coords):
    importance = parameterization.getCanonicalImportance(*coords)
    rankScore = RankingScore(importance=importance)
    showRankingScore(rankScore, fig, ax)
    ax.set_box_aspect(1)
    ax.set_anchor('C')
    ax.axis('off')
    ax.set_title('ROC', fontsize=12, pad=20)


def plot_tile(fig, ax, tpr, fpr):
    priorPos = priorPos_slider_value[0]

    fnr = 1-tpr
    tnr = 1-fpr

    priorNeg = 1-priorPos

    ptn = tnr * priorNeg
    pfp = fpr * priorNeg
    pfn = fnr * priorPos
    ptp = tpr * priorPos

    perf = TwoClassClassificationPerformance(ptn = ptn,
                                            pfp=pfp,
                                            pfn = pfn,
                                            ptp = ptp)
    v_flavor = ValueFlavor(perf, "Value flavor")

    v_tile = ValueTile(name = "Value Tile", 
                       parameterization = parameterization, 
                       flavor = v_flavor,
                       )
    
    a = np.linspace(0, 1, 100)
    b = np.linspace(0, 1, 100)

    a2, b2 = np.meshgrid(a, b, indexing='xy')
    

    res = v_tile(a2, b2)

    ax.imshow(res, extent=extent, origin='lower')
    ax.set_title('Value Tile', fontsize=12, pad=20)

def react_left(fig, ax, event, click_marker_left = click_marker_left, click_marker_right = click_marker_right):

    fig.delaxes(ax[1])
    ax[1] = fig.add_subplot(right_spec)

    plot_roc(fig, ax[1], (event.xdata, event.ydata))

    print("-"*10)
    print(click_marker_left)
    reset_marker(click_marker_left)
    print(click_marker_left)
    print("-"*10)

    click_marker_left[0] = ax[0].plot(event.xdata, event.ydata, 'ro', markersize=10)[0]
    if click_marker_right[0] is not None:
        click_marker_right[0] = ax[1].plot(click_marker_right[0].get_xdata()[0], click_marker_right[0].get_ydata()[0], 'ro', markersize=10)[0]

    fig.canvas.draw_idle()



def react_right(fig, ax, event, click_marker_left = click_marker_left, click_marker_right = click_marker_right):

    fig.delaxes(ax[0])
    ax[0] = fig.add_subplot(left_spec)

    plot_tile(fig, ax[0], event.ydata, event.xdata)

    print("*"*10)
    print(click_marker_right)
    reset_marker(click_marker_right)
    print(click_marker_right)
    print("*"*10)

    click_marker_right[0] = ax[1].plot(event.xdata, event.ydata, 'ro', markersize=10)[0]
    if click_marker_left[0] is not None:
        click_marker_left[0] = ax[0].plot(click_marker_left[0].get_xdata()[0], click_marker_left[0].get_ydata()[0], 'ro', markersize=10)[0]

    fig.canvas.draw_idle()

    print(click_marker_right)
    print(click_marker_left)
    
    

def del_colorbars(fig):
    for a in list(fig.axes):
        if a.get_label() == '<colorbar>':
            fig.delaxes(a)

def on_move(event):
    # Only react when the mouse is inside the left axes and has valid data coords
    if event.xdata is None or event.ydata is None:
        return
    

    if event.inaxes == slider_ax:
        return  # shouldn't be on_prior_slider_change here ?
    
    elif event.inaxes is not ax[0]:
        
        react_right(fig, ax, event)

    else:
        react_left(fig, ax, event)



# Optional: disable updates during pan/zoom
def toolbar_busy():
    tb = getattr(fig.canvas, "toolbar", None)
    return bool(tb and tb.mode)  # don't update while zoom/pan active

def on_move_guarded(event):
    if not toolbar_busy():
        on_move(event)

#cid_move = fig.canvas.mpl_connect('motion_notify_event', on_move_guarded)

slider_ax = fig.add_axes([0.2, 0.05, 0.6, 0.03])  # [left, bottom, width, height]
prior_slider = Slider(
    ax=slider_ax,
    label='Prior Positive Rate',
    valmin=0.01,
    valmax=0.99,
    valinit=priorPos_slider_value[0],
    valstep=0.01,
    orientation='horizontal'
)


def on_prior_slider_change(val):
    if click_marker_left[0] is None and click_marker_right[0] is None:
        return

    priorPos_slider_value[0] = val

    fig.delaxes(ax[0])
    fig.delaxes(ax[1])
    ax[0] = fig.add_subplot(left_spec)
    ax[1] = fig.add_subplot(right_spec)

    if click_marker_right[0] is not None:
        x, y = click_marker_right[0].get_xdata()[0], click_marker_right[0].get_ydata()[0]
        reset_marker(click_marker_right)
        plot_tile(fig, ax[0], y, x)
        click_marker_right[0] = ax[1].plot(x, y, 'ro', markersize=10)[0]

    if click_marker_left[0] is not None:
        x, y = click_marker_left[0].get_xdata()[0], click_marker_left[0].get_ydata()[0]
        reset_marker(click_marker_left)
        plot_roc(fig, ax[1], (x, y))
        click_marker_left[0] = ax[0].plot(x, y, 'ro', markersize=10)[0]

    fig.canvas.draw_idle()

prior_slider.on_changed(on_prior_slider_change)



plt.connect('button_press_event', on_move_guarded)


plt.show()

# TODO fix duplicating colorbar on resize